In [1]:
#!pip install opencv-python

In [2]:
import pandas as pd
import numpy as np
from shutil import copyfile
import glob
import os
import cv2

In [9]:
#!pip install keras

In [7]:
#!pip install tensorflow

In [11]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
#from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.preprocessing import image
from keras.models import load_model

In [ ]:
#image generator, normalizing them to be between 0 and 1
generator_training = ImageDataGenerator(rescale = 1./255)
generator_test = ImageDataGenerator(rescale = 1./255)

In [ ]:
data = pd.read_csv('train.truth.csv')

In [ ]:
data.head()

In [ ]:
train, test = train_test_split(data, test_size=0.25)

In [ ]:
#Dividing the images into folders for their rotation. Necessary for learning
start = False
if (start):
    for i, row in train.iterrows():
        src = 'train/' + row['fn']
        dst = 'images_train/' + row['label'] + '/' + row['fn']
        copyfile(src, dst)
    for i, row in test.iterrows():
        src = 'train/' + row['fn']
        dst = 'images_test/' + row['label'] + '/' + row['fn']
        copyfile(src, dst)

In [ ]:
#Cada pasta o modelo considerará que é uma categoria
base_train = generator_training.flow_from_directory('images_train', target_size=(64,64),
                                                          batch_size=32, class_mode='categorical')

In [ ]:
#Cada pasta o modelo considerará que é uma categoria
base_test = generator_test.flow_from_directory('images_test', target_size=(64,64),
                                                          batch_size=32, class_mode='categorical')

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(64, 64, 3)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(4))
model.add(Activation('softmax'))

In [ ]:
#I used the variable patterns for lack of time
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
if(start):
    model.fit_generator(base_train, steps_per_epoch=36000/32, epochs=5, validation_data=base_test,
                       validation_steps=12000)

In [ ]:
save_dir = os.path.join(os.getcwd(), 'model')
model_name = 'keras_cifar10_trained_model.h5'
model_path = os.path.join(save_dir, model_name)
if(start):    
    # Save model and weights
    if not os.path.isdir(save_dir):
        os.makedirs(save_dir)
    model_path = os.path.join(save_dir, model_name)
    model.save(model_path)
    print('Saved trained model at %s ' % model_path)
else:
    model = load_model('model/my_predict.h5')

# Result

In [ ]:
COLUNAS = [
    'fn',
    'label'
]
resultado = pd.DataFrame(columns = COLUNAS)
resposta = pd.DataFrame(columns = COLUNAS)

In [ ]:
for file in glob.glob("test/*.jpg"):    
    image_test = image.load_img(file, target_size= (64,64))
    inicial = image_test
    image_test = image.img_to_array(image_test)
    image_test /= 255
    image_test = np.expand_dims(image_test, axis = 0)
    previsao = model.predict(image_test)
    if(previsao[0][0].round() == 1):
        #final = inicial.rotate(-90).save(inicial.filename)
        label = 'rotated_left'
    elif (previsao[0][1].round() == 1):
        #final = inicial.rotate(90).save(inicial.filename)  
        label = 'rotated_right'
    elif (previsao[0][3].round() == 1):
        #final = inicial.rotate(180).save(inicial.filename) 
        label = 'upside_down'
    else:
        label = 'upright'
    
    fn = inicial.filename
    fn = fn.split('/')
    resposta = resposta.append({'label':label, 'fn':fn[1]},ignore_index=True,sort=False)
    inicial.close()

In [ ]:
resposta.to_csv('test.preds.csv', index=False)